In [1]:
class Morph:
    """
    1つの形態素を表すクラス
    """

    def __init__(self, surface, base, pos, pos1):
        """
        メンバ変数として表層形（surface），基本形（base），品詞（pos），品詞細分類1（pos1）を持つ.
        """
        self.surface = surface
        self.base = base
        self.pos = pos
        self.pos1 = pos1

    def is_end_of_sentence(self) -> bool: return self.pos1 == '句点'

    def __str__(self) -> str: return 'surface: {}, base: {}, pos: {}, pos1: {}'.format(self.surface, self.base, self.pos, self.pos1)


def make_morph_list(analyzed_file_name: str) -> list:
    """
    係り受け解析済みの文章ファイルを読み込んで、各文をMorphオブジェクトのリストとして表現する
    :param analyzed_file_name 係り受け解析済みの文章ファイル名
    :return list 一つの文章をMorphオブジェクトのリストとして表現したもののリスト
    """
    sentences = []
    sentence = []
    with open(analyzed_file_name, encoding='utf-8') as input_file:
        for line in input_file:
             #* 0 1D 0/1 0.000000
             #吾輩    名詞,代名詞,一般,*,*,*,吾輩,ワガハイ,ワガハイ
             #は      助詞,係助詞,*,*,*,*,は,ハ,ワ
            
            line_list = line.split()
            #['*', '0', '1D', '0/1', '0.000000']
            #['吾輩', '名詞,代名詞,一般,*,*,*,吾輩,ワガハイ,ワガハイ']
            #['は', '助詞,係助詞,*,*,*,*,は,ハ,ワ']
            if (line_list[0] == '*') | (line_list[0] == 'EOS'):
                pass
            else:
                line_list = line_list[0].split(',') + line_list[1].split(',')
                # この時点でline_listはこんな感じ
                # ['始め', '名詞', '副詞可能', '*', '*', '*', '*', '始め', 'ハジメ', 'ハジメ']
                _morph = Morph(surface=line_list[0], base=line_list[7], pos=line_list[1], pos1=line_list[2])
                #printすると__str__通り返ってくるsurface: 一, base: 一, pos: 名詞, pos1: 数
                #格納すると<__main__.Morph object at 0x00000143312DFFC8>, ....
                #何もreturnしてないからね
                sentence.append(_morph)

                if _morph.is_end_of_sentence():
                    #_morphのpos1が句点だったら
                    sentences.append(sentence)
                    sentence = []

    return sentences


morphed_sentences = make_morph_list('neko.txt.cabocha')
#[<__main__.Morph object at 0x0000014330DA8C88>, #文ごと
# 3文目の形態素列を表示
for morph in morphed_sentences[1]:
    print(str(morph))
    #print()とかstr()すると戻る

surface: 名前, base: 名前, pos: 名詞, pos1: 一般
surface: は, base: は, pos: 助詞, pos1: 係助詞
surface: まだ, base: まだ, pos: 副詞, pos1: 助詞類接続
surface: 無い, base: 無い, pos: 形容詞, pos1: 自立
surface: 。, base: 。, pos: 記号, pos1: 句点


In [4]:
class Chunk:
    def __init__(self, morphs: list, dst: str, srcs: str) -> None:
        """
        形態素（Morphオブジェクト）のリスト（morphs），係り先文節インデックス番号（dst），係り元文節インデックス番号のリスト（srcs）をメンバ変数に持つ
        """
        self.morphs = morphs
        self.dst = int(dst.strip("D"))
        self.srcs = int(srcs)

    # 以下は後々使うメソッドです.
    def join_morphs(self) -> str:
        return ''.join([_morph.surface for _morph in self.morphs if _morph.pos != '記号'])
        #句点とかの記号以外を出力
        #surface: 。, base: 。, pos: 記号, pos1: 句点
        #_morph自体はオブジェクトだけど_morph.pos,_morph.surfaceは普通に出てくる
    def has_noun(self) -> bool:
        return any([_morph.pos == '名詞' for _morph in self.morphs])

    def has_verb(self) -> bool:
        return any([_morph.pos == '動詞' for _morph in self.morphs])

    def has_particle(self) -> bool:
        return any([_morph.pos == '助詞' for _morph in self.morphs])

    def has_sahen_connection_noun_plus_wo(self) -> bool:
        """
        「サ変接続名詞+を（助詞）」を含むかどうかを返す.
        """
        for idx, _morph in enumerate(self.morphs):
            if _morph.pos == '名詞' and _morph.pos1 == 'サ変接続' and len(self.morphs[idx:]) > 1 and \
                            self.morphs[idx + 1].pos == '助詞' and self.morphs[idx + 1].base == 'を':
                return True

        return False

    def first_verb(self) -> Morph:
        return [_morph for _morph in self.morphs if _morph.pos == '動詞'][0]

    def last_particle(self) -> list:
        return [_morph for _morph in self.morphs if _morph.pos == '助詞'][-1]

    def pair(self, sentence: list) -> str:
        return self.join_morphs() + '\t' + sentence[self.dst].join_morphs()

    def replace_noun(self, alt: str) -> None:
        """
        名詞の表象を置換する.
        """
        for _morph in self.morphs:
            if _morph.pos == '名詞':
                _morph.surface = alt

    def __str__(self) -> str:
        return 'srcs: {}, dst: {}, morphs: ({})'.format(self.srcs, self.dst, ' / '.join([str(_morph) for _morph in self.morphs]))


def make_chunk_list(analyzed_file_name: str) -> list:
    """
    係り受け解析済みの文章ファイルを読み込んで、各文をChunkオブジェクトのリストとして表現する
    :param analyzed_file_name 係り受け解析済みの文章ファイル名
    :return list 一つの文章をChunkオブジェクトのリストとして表現したもののリスト
    """
    sentences = []
    sentence = []
    _chunk = None
    count=0
    with open(analyzed_file_name, encoding='utf-8') as input_file:
        for line in input_file:
            #
            #count+=1
            #if count==20:
            #    break
            #
            line_list = line.split()
            #print('手始め',line_list)
            if line_list[0] == '*':#文頭
            #    print('上は文頭')
                if _chunk is not None:
            #        print('元からあった_chunk',_chunk)
                    sentence.append(_chunk)#元々あったやつ
            #        print('_chunk入れた',sentence)
                _chunk = Chunk(morphs=[], dst=line_list[2], srcs=line_list[1])
            #    print('文頭の','chunkを定義しなおした',_chunk)
                #文頭のline_list
                #['*', '0', '-1D', '0/0', '0.000000']
            elif line_list[0] == 'EOS':  # End of sentence
            #    print('上はケツ')
                if _chunk is not None:
            #        print('元からあった_chunk',_chunk)
                    sentence.append(_chunk)
            #        print('_chunk入れた',sentence)
                if len(sentence) > 0:
            #        print('sentence',sentence)
                    sentences.append(sentence)
                #リセット
                _chunk = None
                sentence = []
            else:#文頭でもケツでもない
            #    print('上は文頭でもケツでもない')
            #    print('元のlist',line_list)
                line_list = line_list[0].split(',') + line_list[1].split(',')
            #    print('listを定義しなおした',line_list)
                # この時点でline_listはこんな感じ
                # ['始め', '名詞', '副詞可能', '*', '*', '*', '*', '始め', 'ハジメ', 'ハジメ']
                _morph = Morph(surface=line_list[0], base=line_list[7], pos=line_list[1], pos1=line_list[2])
            #    print('_morph',_morph)
                _chunk.morphs.append(_morph)
            #    print('morphに_chunkを足したよ',_chunk)

    return sentences


chunked_sentences = make_chunk_list('neko.txt.cabocha')
print(len(chunked_sentences))
# 3文目の形態素列を表示
for chunk in chunked_sentences[2]:
    
    print(str(chunk))

9210
srcs: 0, dst: 2, morphs: (surface: 名前, base: 名前, pos: 名詞, pos1: 一般 / surface: は, base: は, pos: 助詞, pos1: 係助詞)
srcs: 1, dst: 2, morphs: (surface: まだ, base: まだ, pos: 副詞, pos1: 助詞類接続)
srcs: 2, dst: -1, morphs: (surface: 無い, base: 無い, pos: 形容詞, pos1: 自立 / surface: 。, base: 。, pos: 記号, pos1: 句点)


In [3]:
def is_valid_chunk(_chunk, sentence):
    return _chunk.join_morphs() != '' and _chunk.dst > -1 and sentence[_chunk.dst].join_morphs() != ''
#_chunk.join_morphs() != '':記号以外

paired_sentences = [[chunk.pair(sentence) for chunk in sentence if is_valid_chunk(chunk, sentence)] for sentence in chunked_sentences if len(sentence) > 1]
print(paired_sentences[0:100])
#chumked_sentence:[<__main__.Chunk object at 0x000001AB02B61888>]
#srcs: 0, dst: 2, morphs: (surface: 呼ん, base: 呼ぶ, pos: 動詞, pos1: 自立 / surface: で, base: で, pos: 助詞, pos1: 接続助詞)

[['吾輩は\t猫である'], ['名前は\t無い', 'まだ\t無い'], ['どこで\t生れたか', '生れたか\tつかぬ', 'とんと\tつかぬ', '見当が\tつかぬ'], ['何でも\t薄暗い', '薄暗い\t所で', 'じめじめした\t所で', '所で\t泣いて', 'ニャーニャー\t泣いて', '泣いて\t記憶している', 'いた事だけは\t記憶している'], ['吾輩は\t見た', 'ここで\t始めて', '始めて\t人間という', '人間という\tものを', 'ものを\t見た'], ['しかも\t種族であったそうだ', 'あとで\t聞くと', '聞くと\t種族であったそうだ', 'それは\t種族であったそうだ', '書生という\t人間中で', '人間中で\t種族であったそうだ', '一番\t獰悪な', '獰悪な\t種族であったそうだ'], ['この\t書生というのは', '書生というのは\t話である', '時々\t捕えて', '我々を\t捕えて', '捕えて\t煮て', '煮て\t食うという', '食うという\t話である'], ['しかし\t思わなかった', 'その\t当時は', '当時は\tなかったから', '何という\t考も', '考も\tなかったから', 'なかったから\t思わなかった', '別段\t恐し', '恐し\t思わなかった', 'いとも\t思わなかった'], ['ただ\t載せられて', '彼の\t掌に', '掌に\t載せられて', '載せられて\t持ち上げられた', 'スーと\t持ち上げられた', '持ち上げられた\t時', '時\tフワフワした', '何だか\tフワフワした', 'フワフワした\t感じが', '感じが\tあったばかりである'], ['掌の\t上で', '上で\t落ちついて', '少し\t落ちついて', '落ちついて\t見たのが', '書生の\t顔を', '顔を\t見たのが', '見たのが\t人間という', 'いわゆる\t人間という', '人間という\tものの', 'ものの\t見始であろう'], ['この\t時', '時\tものだと', '妙な\tものだと', 'ものだと\t思った', '思った\t感じが', '感じが\t残っている', '今でも\t残っている'], ['第一毛をもって\t装飾されべきはずの', '装

In [85]:

for sentence in chunked_sentences[:50:]:
    for _chunk in sentence[:10:]:
        print(_chunk.morphs)
        print(_chunk.dst)
        print(sentence[_chunk.dst])
        #print(_chunk.join_morphs())

-1
srcs: 0, dst: -1, morphs: (surface: 一, base: 一, pos: 名詞, pos1: 数)
[<__main__.Morph object at 0x000001AB100F9708>, <__main__.Morph object at 0x000001AB100F9748>]
1
srcs: 1, dst: -1, morphs: (surface: 猫, base: 猫, pos: 名詞, pos1: 一般 / surface: で, base: だ, pos: 助動詞, pos1: * / surface: ある, base: ある, pos: 助動詞, pos1: * / surface: 。, base: 。, pos: 記号, pos1: 句点)
[<__main__.Morph object at 0x000001AB100F9788>, <__main__.Morph object at 0x000001AB100F97C8>, <__main__.Morph object at 0x000001AB100F9808>, <__main__.Morph object at 0x000001AB100F9888>]
-1
srcs: 1, dst: -1, morphs: (surface: 猫, base: 猫, pos: 名詞, pos1: 一般 / surface: で, base: だ, pos: 助動詞, pos1: * / surface: ある, base: ある, pos: 助動詞, pos1: * / surface: 。, base: 。, pos: 記号, pos1: 句点)
[<__main__.Morph object at 0x000001AB100F98C8>, <__main__.Morph object at 0x000001AB100F9948>]
2
srcs: 2, dst: -1, morphs: (surface: 無い, base: 無い, pos: 形容詞, pos1: 自立 / surface: 。, base: 。, pos: 記号, pos1: 句点)
2
srcs: 2, dst: -1, morphs: (surface: 無い, base: 無い